In [8]:
# fisrt install these packages : python-telegram-bot==13.7 telethon utils requests config

from telegram import Update , ReplyKeyboardMarkup
from telegram.ext import Updater, CommandHandler, CallbackContext,ConversationHandler, MessageHandler, Filters
from telegram.error import TelegramError
import requests

In [9]:
class ChatSession:
    def __init__(self, api_key, model):
        self.api_key = api_key
        self.model = model
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        self.history = [
            {"role": "system", "content": "Answer as concisely as possible, make sure your answer does not exceed 4000 characters, and respond in the language in which the question is asked."}
        ]

    def chat_request(self, prompt):
        self.history.append({"role": "user", "content": prompt})

        data = {
            "model": self.model,
            "messages": self.history,
            "stream": False
        }

        try:
            response = requests.post(
                "https://openrouter.ai/api/v1/chat/completions",
                headers=self.headers,
                json=data
            )
            if response.status_code == 200:
                response_json = response.json()
                if "choices" in response_json:
                    model_response = response_json["choices"][0]["message"]["content"]
                    self.history.append({"role": "assistant", "content": model_response})
                    return model_response
                else:
                    print("Response hasn't 'choices' field.")
            else:
                print(f"API Error: {response.status_code} - {response.text}")
        except requests.RequestException as e:
            print(f"Request Error: {e}")
        return ""

In [12]:
TOKEN = ""   # Telgram bot Token

API_KEY_deepseek , MODEL_deepseek = "" , "" # You can take any LLM API from openrouter.ai. Here, I chose Deepseek and Gemini models
API_KEY_gemini , Model_gemini= "" , ""


API , Model = [API_KEY_deepseek , API_KEY_gemini] ,[MODEL_deepseek , Model_gemini]

chat_list , chat_number= {} , ""

def start(update , context):
  update.message.reply_text("What? press /chat_session or Fuck yourself")


def chat_history(update , context):
  markup = ReplyKeyboardMarkup([["/chat_new"]+[f"/chat_{i+1}" for i in range(len(chat_list))]],

                               one_time_keyboard = True)
  update.message.reply_text("Choose your chat:",
                            reply_markup=markup)


def chat_index(update , context):
  global chat_number
  if update.message.text == "/chat_new":
    update.message.reply_text("starting new chat..." )

    reply_keyboard = [["ChatGPT", "DeepSeek"]]
    markup = ReplyKeyboardMarkup(reply_keyboard,
                                 one_time_keyboard = True)
    update.message.reply_text("Choose a Model:",
                              reply_markup=markup)
    i = len(chat_list)
    chat_number = f"chat_{i+1}"
  else:
    chat_number = update.message.text.replace("/", "")
    update.message.reply_text(f"You're in {chat_number}" )


def handle_message(update , context):
  global chat_list
  global API
  global Model
  global chat_number
  choice = update.message.text

  if choice == "ChatGPT":
      update.message.reply_text("I was cheating, the model is Gemini :) ask your dumb question")
      API_KEY_gemini , Model_gemini = API[1] , Model[1]
      chat_list[chat_number] = ChatSession(api_key = API_KEY_gemini ,
                                           model = Model_gemini )

  elif choice == "DeepSeek":
      update.message.reply_text("You're supporting chinese communism...ask your dumb question")
      API_KEY_deepseek , MODEL_deepseek = API[0] , Model[0]
      chat_list[chat_number] =  ChatSession(api_key = API_KEY_deepseek ,
                                           model = MODEL_deepseek )

  else:
      prompt = choice
      thinking_massage = update.message.reply_text("Thinking...")
      respond = chat_list[chat_number].chat_request(prompt)
      thinking_massage.edit_text(respond)

updater = Updater(TOKEN, use_context=True)
dispatch = updater.dispatcher

dispatch.add_handler(CommandHandler("start" , start))

dispatch.add_handler(CommandHandler("chat_session" , chat_history))

dispatch.add_handler(CommandHandler("chat_new", chat_index))

for i in range(100):  # Maximum number of Chat sessions
  dispatch.add_handler(CommandHandler(f"chat_{i+1}", chat_index))

dispatch.add_handler(MessageHandler(Filters.text , handle_message))

updater.start_polling()
updater.idle()